# 1: Imports and Environment Check


In [1]:
import cv2
import numpy as np
import time

# Verify versions - IBM expects you to know your environment
print(f"OpenCV Version: {cv2.__version__}")
print(f"NumPy Version: {np.__version__}")
print("If you see errors here, you didn't install 'opencv-python' or 'numpy'. Fix it.")

OpenCV Version: 4.13.0
NumPy Version: 2.2.6
If you see errors here, you didn't install 'opencv-python' or 'numpy'. Fix it.


In [ ]:

# cap = cv2.VideoCapture(0)

# # 1. Read ONE frame first to get the camera's native size
# ret, frame = cap.read()
# if ret:
#     native_height, native_width, _ = frame.shape
# else:
#     native_height, native_width = 480, 640 # Fallback if camera fails

# # 2. Create the window with the "Normal" flag (Allows resizing)
# window_name = 'Validating Window Size'
# cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# # 3. Force the INITIAL size to match the camera's native resolution
# # This ensures it doesn't open "huge", but you can still stretch it later.
# cv2.resizeWindow(window_name, native_width, native_height)

: 

# 2: Data Extraction & Resolution

In [2]:

# We re-initialize the camera for this specific isolated task
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- THE LOGIC: DATA EXTRACTION ---
    # Shape returns a tuple: (Height, Width, Channels)
    # This proves the image is just a 3D Matrix
    
    height, width, channels = frame.shape
    total_pixels = height * width
    
    # Visualization: writing the raw data on the screen
    font = cv2.FONT_HERSHEY_SIMPLEX
    info_text = f"Matrix Shape: {height}x{width} | Channels: {channels}"
    cv2.putText(frame, info_text, (10, 50), font, 0.7, (0, 255, 0), 2)
    
    cv2.imshow('Data Extraction Cell', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 3. Grayscale Conversions

In [3]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- THE LOGIC: COLOR CONVERSION ---
    # Convert BGR (OpenCV default) to Grayscale (1 Channel)
    # This effectively "flattens" the 3D matrix into a 2D matrix
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Visualization
    cv2.imshow('Original BGR', frame)
    cv2.imshow('Converted Grayscale', gray_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 4. BGR to RGb Visualization

In [4]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- THE LOGIC: CHANNEL SWAPPING ---
    # We convert BGR to RGB. 
    # Since cv2.imshow EXPECTS BGR, displaying this 'rgb_frame' will look weird.
    # This proves that the data bytes have physically moved positions.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    cv2.putText(rgb_frame, "Colors Swapped (Blue is Red)", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Visualization
    cv2.imshow('Standard BGR (Correct)', frame)
    cv2.imshow('RGB Data displayed as BGR (Swapped)', rgb_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 5. FPS Calculation

In [5]:
# CELL 5: FPS Calculation Logic
cap = cv2.VideoCapture(0)

# Initialize variables specifically for this loop
prev_frame_time = 0
new_frame_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- THE LOGIC: MATH ---
    new_frame_time = time.time()
    
    # Avoid division by zero error on the very first frame
    if new_frame_time - prev_frame_time > 0:
        fps = 1 / (new_frame_time - prev_frame_time)
    else:
        fps = 0
        
    prev_frame_time = new_frame_time
    fps_int = int(fps)

    # Visualization
    cv2.putText(frame, f"FPS: {fps_int}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    cv2.imshow('FPS Calculation Cell', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 6: Live Matrix/Pixel Inspection

In [6]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- THE LOGIC: MATRIX INDEXING ---
    # We grab the pixel at row 100, column 100
    # The output is an array [Blue, Green, Red] values (0-255)
    px_val = frame[100, 100]
    
    # Drawing a circle so you can see WHICH pixel we are reading
    cv2.circle(frame, (100, 100), 5, (0, 0, 255), 2)
    
    # Visualization
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, f"Pixel(100,100) Data: {px_val}", (10, 150), font, 0.6, (255, 255, 0), 2)
    
    cv2.imshow('Matrix Inspector', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 7 : Blue Object Tracker

In [7]:
# CELL 7: Color Isolation (Blue Tracker)
cap = cv2.VideoCapture(0)

print("Show a blue object to the camera!")

while True:
    ret, frame = cap.read()
    if not ret: break

    # --- THE LOGIC: HSV MASKING ---
    # 1. Convert BGR to HSV (Hue Saturation Value)
    # This is the industry standard for color tracking
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 2. Define the range of Blue color in HSV
    # Note: In OpenCV, Hue is 0-179. Blue is roughly 110-130.
    lower_blue = np.array([100, 50, 50])   # Lower limit (Dark blueish)
    upper_blue = np.array([140, 255, 255]) # Upper limit (Bright pure blue)

    # 3. Create a Mask
    # A mask is a binary image: White (255) where the color is, Black (0) elsewhere
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # 4. Bitwise AND
    # We take the original frame and "AND" it with the mask
    # It essentially "cuts out" everything that isn't blue
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Visualization - Showing all 3 stages of processing
    cv2.imshow('1. Original', frame)
    cv2.imshow('2. The Mask (Binary)', mask)
    cv2.imshow('3. Final Result', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Show a blue object to the camera!
